### This notebook is used to clean and process the yearly raw data into a Pandas Dataframe

Explanation of the steps can be found in the **BIXI** notebook

In [2]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
FILE_EXPORT_NAME = 'bixi_processed_2016'

# Load BIXI data

In [4]:
zf = ZipFile('./data/bixi/biximontrealrentals2016-912f00.zip')

Some of the year have a different storage convention

In [5]:
zf.namelist()

['BixiMontrealRentals2016/',
 'BixiMontrealRentals2016/OD_2016-04.csv',
 'BixiMontrealRentals2016/OD_2016-05.csv',
 'BixiMontrealRentals2016/OD_2016-06.csv',
 'BixiMontrealRentals2016/OD_2016-07.csv',
 'BixiMontrealRentals2016/OD_2016-08.csv',
 'BixiMontrealRentals2016/OD_2016-09.csv',
 'BixiMontrealRentals2016/OD_2016-10.csv',
 'BixiMontrealRentals2016/OD_2016-11.csv',
 'BixiMontrealRentals2016/Stations_2016.csv']

In [4]:
bixi_04 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_05 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_06 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_07 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_08 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_09 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_10 = pd.read_csv(zf.open('BixiMontrealRentals2016/OD_2016-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_stations = pd.read_csv(zf.open('BixiMontrealRentals2016/Stations_2016.csv'), dtype={'name': 'str'})

In [5]:
bixi_stations

,code,name,latitude,longitude
0,6315,Lacombe / de la Côte-des-Neiges,45.497,-73.623
1,6019,Berri / Sherbrooke,45.518,-73.567
2,6347,Métro St-Michel (Shaughnessy / St-Michel),45.559,-73.600
3,6219,de l'Hôtel-de-Ville / Roy,45.517,-73.574
4,6211,Roy / St-Laurent,45.516,-73.576
...,...,...,...,...
460,6754,Eadie/Dubois,45.458,-73.591
461,6005,Metcalfe / Square Dorchester,45.500,-73.571
462,6383,Bourbonnière / du Mont-Royal,45.554,-73.560
463,6359,Parc Plage,45.503,-73.528


In [6]:
bixi_array = [bixi_04, bixi_05, bixi_06, bixi_07, bixi_08, bixi_09, bixi_10]

# Load weather data

In [7]:
# Note: Use Date/Time column instead of Time column
weather_04 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_04-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_05 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_05-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_06 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_06-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_07 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_07-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_08 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_08-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_09 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_09-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})
weather_10 = pd.read_csv('./data/weather/2016/en_climate_hourly_QC_7025251_10-2016_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'str', 'Weather': 'str'})

In [8]:
weather_array = [weather_04, weather_05, weather_06, weather_07, weather_08, weather_09, weather_10]

# Merge

In [9]:
bixi = pd.concat(bixi_array)

In [10]:
weather = pd.concat(weather_array)

In [11]:
# Some years used 'Code' and other 'code'
bixi = bixi.merge(
    bixi_stations, 
    left_index=True, 
    left_on='start_station_code', 
    right_on='code').drop('code', 1).rename(
    {'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, 
    axis=1).merge(
    bixi_stations, 
    left_index=True, 
    left_on='end_station_code', 
    right_on='code').drop('code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)

# Feature Extract

In [12]:
weather = weather[[
    'Date/Time', 
    'Year',
    'Month',
    'Day',
    'Temp (°C)', 
    'Dew Point Temp (°C)',
    'Rel Hum (%)',
    'Wind Dir (10s deg)',
    'Wind Spd (km/h)',
    'Visibility (km)', 
    'Stn Press (kPa)', 
    'Weather']]

weather['Hour'] = weather['Date/Time'].dt.hour
weather['Minute'] = weather['Date/Time'].dt.minute
weather['Second'] = weather['Date/Time'].dt.second
weather['Day_of_year'] = weather['Date/Time'].dt.dayofyear

# Hot Encode Weather

In [13]:
hot_encode = weather['Weather'].str.get_dummies(',')

In [14]:
weather = pd.concat([weather, hot_encode], axis=1, sort=False).drop('Weather', axis=1)

# Missing values

In [15]:
weather.isnull().sum()

Date/Time                0
Year                     0
Month                    0
Day                      0
Temp (°C)                1
Dew Point Temp (°C)      1
Rel Hum (%)              1
Wind Dir (10s deg)       7
Wind Spd (km/h)          1
Visibility (km)          1
Stn Press (kPa)          1
Hour                     0
Minute                   0
Second                   0
Day_of_year              0
Blowing Snow             0
Clear                    0
Cloudy                   0
Drizzle                  0
Fog                      0
Heavy Rain               0
Ice Pellets              0
Mainly Clear             0
Moderate Rain            0
Moderate Rain Showers    0
Mostly Cloudy            0
Rain                     0
Rain Showers             0
Snow                     0
Snow Showers             0
Thunderstorms            0
dtype: int64

In [16]:
weather = weather.fillna(method='ffill') 
weather.isnull().sum()

Date/Time                0
Year                     0
Month                    0
Day                      0
Temp (°C)                0
Dew Point Temp (°C)      0
Rel Hum (%)              0
Wind Dir (10s deg)       0
Wind Spd (km/h)          0
Visibility (km)          0
Stn Press (kPa)          0
Hour                     0
Minute                   0
Second                   0
Day_of_year              0
Blowing Snow             0
Clear                    0
Cloudy                   0
Drizzle                  0
Fog                      0
Heavy Rain               0
Ice Pellets              0
Mainly Clear             0
Moderate Rain            0
Moderate Rain Showers    0
Mostly Cloudy            0
Rain                     0
Rain Showers             0
Snow                     0
Snow Showers             0
Thunderstorms            0
dtype: int64

# Sum BIXI Trips

In [17]:
temp = bixi
station_group = temp.groupby('start_station_code')

In [18]:
hour_trips_dict = {}

with tqdm(total=len(station_group)) as pbar:
    for station_code, df in station_group:
        # Randomly appending "start_year" since all columns return the same number. Renaming it after.
        hour_trips = pd.DataFrame(df.set_index('start_date').resample('H').count()['start_name']).rename(columns={'start_name': 'hour_trip_count'})
        hour_trips_dict[station_code] = hour_trips
        pbar.update(1)

100%|██████████| 465/465 [00:03<00:00, 141.36it/s]


In [19]:
hour_trips_dict[6712].head(20)

,hour_trip_count
start_date,
2016-04-15 07:00:00,1
2016-04-15 08:00:00,1
2016-04-15 09:00:00,0
2016-04-15 10:00:00,0
2016-04-15 11:00:00,0
2016-04-15 12:00:00,0
2016-04-15 13:00:00,0
2016-04-15 14:00:00,0
2016-04-15 15:00:00,0


# Merge weather data with bike trips

In [20]:
for i in hour_trips_dict:
    df = hour_trips_dict[i]
    df = df.merge(weather, left_on='start_date', right_on='Date/Time')
    hour_trips_dict[i] = df

In [21]:
hour_trips_dict[6712]

,hour_trip_count,Date/Time,Year,Month,Day,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),...,Ice Pellets,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Snow,Snow Showers,Thunderstorms
0,1,2016-04-15 07:00:00,2016,4,15,2.200,-5.200,58.000,3.000,21.000,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-04-15 08:00:00,2016,4,15,4.700,-4.300,52.000,2.000,18.000,...,0,0,0,0,0,0,0,0,0,0
2,0,2016-04-15 09:00:00,2016,4,15,7.300,-3.600,46.000,1.000,16.000,...,0,0,0,0,0,0,0,0,0,0
3,0,2016-04-15 10:00:00,2016,4,15,9.100,-5.200,36.000,5.000,18.000,...,0,0,0,0,0,0,0,0,0,0
4,0,2016-04-15 11:00:00,2016,4,15,10.500,-5.900,31.000,5.000,16.000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,0,2016-10-31 09:00:00,2016,10,31,3.400,-3.000,63.000,33.000,14.000,...,0,0,0,0,0,0,0,0,0,0
4779,0,2016-10-31 10:00:00,2016,10,31,3.900,-3.200,60.000,33.000,18.000,...,0,0,0,0,1,0,0,0,0,0
4780,0,2016-10-31 11:00:00,2016,10,31,4.200,-3.300,58.000,25.000,10.000,...,0,0,0,0,0,0,0,0,0,0
4781,0,2016-10-31 12:00:00,2016,10,31,4.100,-3.900,56.000,29.000,12.000,...,0,0,0,0,0,0,0,0,0,0


# Export

We are exporting the dictionary of DataFrames into a .pbz2 file rather than a regular Pickle file to save disk space

In [22]:
import pickle
import _pickle as cPickle
import bz2

In [23]:
def compressed_pickle(title, data):
    with bz2.BZ2File('./data/output/' + title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)

In [24]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [27]:
compressed_pickle(FILE_EXPORT_NAME, hour_trips_dict)

Importing the data back and making sure that we can use it

In [28]:
import_hour_trips_dict = decompress_pickle('./data/output/%s.pbz2' % FILE_EXPORT_NAME)

In [29]:
import_hour_trips_dict[6712]

,hour_trip_count,Date/Time,Year,Month,Day,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),...,Ice Pellets,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Snow,Snow Showers,Thunderstorms
0,1,2016-04-15 07:00:00,2016,4,15,2.200,-5.200,58.000,3.000,21.000,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-04-15 08:00:00,2016,4,15,4.700,-4.300,52.000,2.000,18.000,...,0,0,0,0,0,0,0,0,0,0
2,0,2016-04-15 09:00:00,2016,4,15,7.300,-3.600,46.000,1.000,16.000,...,0,0,0,0,0,0,0,0,0,0
3,0,2016-04-15 10:00:00,2016,4,15,9.100,-5.200,36.000,5.000,18.000,...,0,0,0,0,0,0,0,0,0,0
4,0,2016-04-15 11:00:00,2016,4,15,10.500,-5.900,31.000,5.000,16.000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,0,2016-10-31 09:00:00,2016,10,31,3.400,-3.000,63.000,33.000,14.000,...,0,0,0,0,0,0,0,0,0,0
4779,0,2016-10-31 10:00:00,2016,10,31,3.900,-3.200,60.000,33.000,18.000,...,0,0,0,0,1,0,0,0,0,0
4780,0,2016-10-31 11:00:00,2016,10,31,4.200,-3.300,58.000,25.000,10.000,...,0,0,0,0,0,0,0,0,0,0
4781,0,2016-10-31 12:00:00,2016,10,31,4.100,-3.900,56.000,29.000,12.000,...,0,0,0,0,0,0,0,0,0,0


In [30]:
len(import_hour_trips_dict)

465

In [31]:
len(hour_trips_dict)

465